In [2]:
import pandas as pd

def isNaN(string):
    return string != string


print("Fetching required fields from joined dataset initiated...")

combined_lines_rd = pd.read_csv("C:/Users/Admin/Mini_Project1/24[7]_Project/core_part/processed_text/combined_lines_chars.csv", header=0, index_col=False)

listofrows = combined_lines_rd.to_numpy().tolist()

dialogues = []
gender = []

for rows in listofrows:
    if(not isNaN(rows[3])):
        dialogues.append(rows[3])
        try:
            if(isNaN(rows[4])):
                gender.append("?")
            else:
                gender.append(rows[4].lower())
        except AttributeError:
            print("Invalid Gender class available in dataset")

print("Fetching required fields from joined dataset completed.")

Fetching required fields from joined dataset initiated...
Fetching required fields from joined dataset completed.


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english')) 


def remove_punct(strin):
    punct_str = '!"#$%&\'()*+,-./:;?@[\]^_`{|}~'
    restr = ""
    for letter in strin:
        if(letter not in punct_str):
            restr = restr+letter
    return restr

In [6]:

import numpy as np

docs = []
class_labels = []

print("Cleaning the dataset initiated...")

max_length_in_words = 0

ind=0
for lines in dialogues:
    if(gender[ind] is not '?'):
        clean_data_sent = remove_punct(lines)
        clean_data_words = clean_data_sent.split()
        pure_sentence = ""
        for eachword in clean_data_words:
            if(len(eachword) > 1 and eachword.lower() not in stop_words):
                pure_sentence+=eachword.lower()+" "
        if(len(pure_sentence.split())>50):
            temp_actual = pure_sentence.split()
            while (len(temp_actual)>50):
                sentence = ""
                temp_list = temp_actual[:50]
                for temp_words in temp_list:
                    sentence += temp_words + " "
                temp_actual = temp_actual[50:]
                docs.append(sentence)
                if(gender[ind] == 'm'):
                    class_labels.append(1)
                else:
                    class_labels.append(0)
            if(len(temp_actual)>0):
                sentence = ""
                temp_list = temp_actual[:50]
                for temp_words in temp_list:
                    sentence += temp_words + " "
                docs.append(sentence)
                if(gender[ind] == 'm'):
                    class_labels.append(1)
                else:
                    class_labels.append(0)
        else:
            docs.append(pure_sentence)
            if(gender[ind] == 'm'):
                class_labels.append(1)
            else:
                class_labels.append(0)
        #if(len(pure_sentence.strip())>0 and len(pure_sentence.split())>=10):
         #   docs.append(pure_sentence.strip())
          #  if(len(pure_sentence.split())>max_length_in_words):
           #     max_length_in_words = len(pure_sentence.split())
           # if(gender[ind] == 'm'):
           #     class_labels.append(1)
           # else:
           #     class_labels.append(0)
    ind+=1

print("Cleaning the dataset completed.")

#ind=0
#for rows in actual_data:
#    print("[",len(rows),"]:",rows," - ",class_labels[ind])
#    ind+=1

Cleaning the dataset initiated...
Cleaning the dataset completed.


In [7]:
import tensorflow as tffrom 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv1D
from tensorflow.keras.layers import Embedding

print("Assigning labels to fit in CNN...")

labels = np.array(class_labels)
vocab_size = 2000

print("Encoding given words to values...")

encoded_docs = [one_hot(d, vocab_size) for d in docs]
#print(encoded_docs)
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)
print("Defining the CNN model. Summary is provided below")
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Dropout(0.2))
model.add(Conv1D(filters=1,kernel_size=3,padding='valid', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
print("Fitting the model with data...")
model.fit(padded_docs, labels, epochs=100, verbose=1)
# evaluate the model
print("Calculating accuracy...")
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

Assigning labels to fit in CNN...
Encoding given words to values...
Defining the CNN model. Summary is provided below
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 8)             16000     
_________________________________________________________________
dropout (Dropout)            (None, 50, 8)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 48, 1)             25        
_________________________________________________________________
flatten (Flatten)            (None, 48)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 49        
Total params: 16,074
Trainable params: 16,074
Non-trainable params: 0
_________________________________________________________________


303540/303540 [==============================] - 53s 174us/sample - loss: 0.6734 - accuracy: 0.5652
Epoch 64/100
303540/303540 [==============================] - 53s 175us/sample - loss: 0.6736 - accuracy: 0.5653
Epoch 65/100
303540/303540 [==============================] - 52s 173us/sample - loss: 0.6735 - accuracy: 0.5657
Epoch 66/100
303540/303540 [==============================] - 53s 173us/sample - loss: 0.6735 - accuracy: 0.5663
Epoch 67/100
303540/303540 [==============================] - 53s 175us/sample - loss: 0.6735 - accuracy: 0.5654
Epoch 68/100
303540/303540 [==============================] - 52s 173us/sample - loss: 0.6735 - accuracy: 0.5655
Epoch 69/100
303540/303540 [==============================] - 52s 171us/sample - loss: 0.6735 - accuracy: 0.5650
Epoch 70/100
303540/303540 [==============================] - 52s 172us/sample - loss: 0.6735 - accuracy: 0.5661
Epoch 71/100
303540/303540 [==============================] - 53s 174us/sample - loss: 0.6735 - accuracy: 0.5

In [8]:
model.save("gender_cnn_model1.model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: gender_cnn_model1.model\assets


In [7]:
#Prediction_sample


docs=['I looked for you back at the party but you always seemed to be occupied']
vocab_size = 2000
genders = ["Female","Male"]
encoded_docs = [one_hot(d, vocab_size) for d in docs]
# pad documents to a max length of 4 words
max_length = max_length_in_words
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
result = model.predict(padded_docs)
ind = 0
for row in result:
    if(row[0]<0.5):
        out = 0
    else:
        out = 1
    ind+=1

print(docs[0]," - ",genders[out])

I looked for you back at the party but you always seemed to be occupied  -  Female
